In [1]:
import requests

In [2]:
URL = "https://thor-galabau.com/"

In [3]:
# request html content
html = requests.get(URL).content

In [4]:
# read the text of the whole page
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)
body_raw = soup.get_text().strip()

In [5]:
# format the text
import re
body_raw = re.compile(r'\n').sub(' ', body_raw)
body_raw = re.compile(r'\t').sub(' ', body_raw)
body_text = body_raw.strip()

In [10]:
body_text

'THOR Garten- & Landschaftsbau • Experten für Ihre Gartengestaltung.                                                                   Leistungen Über uns Inspirationen Karriere Kontakt    Menü  Leistungen Über uns Inspirationen Karriere Kontakt                                                           THOR Garten- & Landschaftsbau     Ihr Gartenbau-Partner im Rhein-Main-Gebiet.     Der grüne Hammer für Ihren Garten.                 Was bieten wir Ihnen im Garten- und Landschaftsbau?     Als Experte für Garten- und Landschaftsbau planen, gestalten und setzen wir Ihre Gartenträume in die Wirklichkeit um. Wir sind für Sie der ideale Partner in Frankfurt und dem kompletten Rhein-Main-Gebiet voller Erfahrung und Fachwissen rund um das Thema Gartengestaltung und Gartenbau.Wir kennen die besonderen Eigenschaften und Bedürfnisse der verschiedenen Blumen, Gräser, Stauden, Sträucher und Bäume, die wir gemeinsam für Ihren Garten auswählen und fachgerecht am richtigen Standort verpflanzen.Als reg

In [6]:
import os
os.chdir('/Users/siegstedt/projects/kreativkopf')

In [7]:
from src.prepare import generate_keywords_from_text
number_of_keywords = 25
generate_keywords_from_text(body_text, number_of_keywords)

[('thor', 0.1667484308864522),
 ('garten', 0.1667484308864522),
 ('gartenbau', 0.15047752587332247),
 ('partner', 0.15047752587332247),
 ('gartengestaltung', 0.14090322679853273),
 ('leistungen', 0.14090322679853273),
 ('kontakt', 0.14022072550356757),
 ('hammer', 0.12776765562815037),
 ('gruen', 0.12620212175468762),
 ('ersten', 0.12541898661930337),
 ('vorstellung', 0.11853436974544732),
 ('unternehmen', 0.11819565531715455),
 ('formular', 0.11348371883478861),
 ('setzen', 0.11169283749064302),
 ('landschaftsbau', 0.11104953827935503),
 ('fachgerechte', 0.10890435307762357),
 ('gartenpflege', 0.10671801844173998),
 ('unverbindliches', 0.10560055179103481),
 ('fachwissen', 0.10473755828695967),
 ('lassen', 0.10469368933818268),
 ('moechten', 0.10468444061480181),
 ('rueckruf', 0.1027662494729944),
 ('ideen', 0.1025021268896132),
 ('gartenplanung', 0.10149862611089588),
 ('alles', 0.09973882294768403)]

In [19]:
# explicit check for locations in text
for i in ['Frankfurt', 'Mainhausen']:
    if i in body_text:
        print('Found location:',i)

Found location: Frankfurt
Found location: Mainhausen
